## Estimador API

In [1]:
import tensorflow as tf
from tensorflow import estimator
from sklearn.datasets import load_wine

In [2]:
vino = load_wine()

In [3]:
print(vino['DESCR'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [4]:
carateristicas = vino['data']
objetivo = vino['target']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(carateristicas, objetivo, test_size=0.3)

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
normalizador = MinMaxScaler()

In [12]:
X_train_normalizado = normalizador.fit_transform(X_train)
X_test_normalizado = normalizador.transform(X_test)

In [14]:
X_train

array([[1.334e+01, 9.400e-01, 2.360e+00, ..., 1.020e+00, 1.930e+00,
        7.500e+02],
       [1.269e+01, 1.530e+00, 2.260e+00, ..., 9.600e-01, 2.060e+00,
        4.950e+02],
       [1.229e+01, 3.170e+00, 2.210e+00, ..., 1.420e+00, 2.830e+00,
        4.060e+02],
       ...,
       [1.332e+01, 3.240e+00, 2.380e+00, ..., 5.500e-01, 1.620e+00,
        6.500e+02],
       [1.430e+01, 1.920e+00, 2.720e+00, ..., 1.070e+00, 2.650e+00,
        1.280e+03],
       [1.350e+01, 1.810e+00, 2.610e+00, ..., 1.120e+00, 3.820e+00,
        8.450e+02]])

In [15]:
X_train_normalizado.shape

(124, 13)

In [16]:
columnas_caracteristicas = [tf.feature_column.numeric_column('x', shape=[13])]

In [17]:
modelo = estimator.DNNClassifier(hidden_units=[20,20,20], feature_columns=columnas_caracteristicas, n_classes=3, optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_tf_random_seed': None, '_is_chief': True, '_train_distribute': None, '_num_ps_replicas': 0, '_service': None, '_task_id': 0, '_evaluation_master': '', '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe7041f9588>, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_model_dir': '/tmp/tmpyigyn0c0', '_device_fn': None, '_master': '', '_session_config': None, '_num_worker_replicas': 1}


In [18]:
funcion_estrada = estimator.inputs.numpy_input_fn(x={'x':X_train_normalizado}, y=y_train, shuffle=True, batch_size=10, num_epochs=10)

In [19]:
modelo.train(input_fn=funcion_estrada, steps=600)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpyigyn0c0/model.ckpt.
INFO:tensorflow:loss = 10.829147, step = 1
INFO:tensorflow:global_step/sec: 205.771
INFO:tensorflow:loss = 6.3226967, step = 101 (0.490 sec)
INFO:tensorflow:Saving checkpoints for 124 into /tmp/tmpyigyn0c0/model.ckpt.
INFO:tensorflow:Loss for final step: 1.4868431.


In [20]:
funcion_evaluacion = estimator.inputs.numpy_input_fn(x={'x':X_test_normalizado}, shuffle=False)

In [21]:
predicciones = list(modelo.predict(input_fn=funcion_evaluacion))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpyigyn0c0/model.ckpt-124
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [22]:
predicciones_finales = [p['class_ids'][0] for p in predicciones]

In [23]:
predicciones_finales

[1,
 1,
 2,
 0,
 1,
 2,
 0,
 0,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 2,
 1,
 1,
 0,
 2,
 1,
 2,
 1,
 0,
 1,
 1,
 0,
 1,
 2,
 1,
 0,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 0,
 0]

In [24]:
from sklearn.metrics import confusion_matrix, classification_report

In [25]:
print(classification_report(y_test, predicciones_finales))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97        18
           1       0.88      0.95      0.91        22
           2       0.92      0.86      0.89        14

   micro avg       0.93      0.93      0.93        54
   macro avg       0.93      0.92      0.92        54
weighted avg       0.93      0.93      0.93        54

